# Estimating Physics Parameters

This notebook shows how the estimation phase of Spyral works. To use this notebook the first two phases (point cloud and cluster) of Spyral *must* have been run on the data. Once clusters have been identified, the next step is to estimate the physics parameters which will be feed to the solver of choice (either ODE or Kalman filter). The main points are the initial orientation and energy of the particle, as well as the species of the particle. 

First we import our code

In [ ]:
from spyral.core.cluster import Cluster
from spyral.core.estimator import estimate_physics
from spyral.geometry.circle import generate_circle_points
from spyral.core.run_stacks import form_run_string
from spyral import EstimateParameters, DetectorParameters

from pathlib import Path
import h5py as h5
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Now that we have our modules, we need to specifiy our configuration as per usual

In [ ]:
# Set some parameters
workspace_path = Path("/path/to/your/workspace/")

est_params = EstimateParameters(
    min_total_trajectory_points=30, smoothing_factor=1.0
)

det_params = DetectorParameters(
    magnetic_field=2.85,
    electric_field=45000.0,
    detector_length=1000.0,
    beam_region_radius=25.0,
    micromegas_time_bucket=10.0,
    window_time_bucket=560.0,
    get_frequency=6.25,
    garfield_file_path=Path("/path/to/some/garfield.txt"),
    do_garfield_correction=False,
)

cluster_path = workspace_path / "Cluster" # This may change if you add custom phases!


For this phase, we also need some storage to hold the results. The storage is in the form of a dictionary of lists

In [ ]:
results: dict[str, list] = {
    'event': [], 
    'cluster_index': [], 
    'cluster_label': [], 
    'ic_amplitude': [], 
    'ic_centroid': [], 
    'ic_integral': [], 
    'ic_multiplicity': [], 
    'vertex_x': [], 
    'vertex_y': [], 
    'vertex_z': [],
    'center_x': [], 
    'center_y': [], 
    'center_z': [], 
    'polar': [], 
    'azimuthal': [],
    'brho': [], 
    'dEdx': [], 
    'dE': [], 
    'arclength': [], 
    'direction': []
}

From our workspace we can then request the file for our clusters, and read in a random cluster from the file

In [ ]:
run_number = 16
cluster_file_path = cluster_path / f"{form_run_string(run_number)}.h5"
cluster_file = h5.File(cluster_file_path, "r")
cluster_group = cluster_file['cluster']
min_event = cluster_group.attrs['min_event']
max_event = cluster_group.attrs['max_event']
event_group = None
event = 0
nclusters = 0
while(True):
    event = np.random.randint(min_event, max_event)
    # event = 10968
    try:
        event_group = cluster_group[f'event_{event}']
        nclusters = event_group.attrs['nclusters']
        if nclusters != 0:
            break
    except:
        continue
local_cluster = None
cluster_index = np.random.randint(0, nclusters)
# cluster_index = 0
local_cluster = event_group[f'cluster_{cluster_index}']

print(f'event: {event}')
print(f'cluster index: {cluster_index}')
cluster = Cluster(event, local_cluster.attrs['label'], local_cluster['cloud'][:].copy())
print(f"cluster size: {len(cluster.data)}")

With our cluster selected and loaded, we can now send it, along with some configuration paramters, through the estimator code.

In [ ]:
estimate_physics(
    cluster_index, 
    cluster, 
    event_group.attrs['ic_amplitude'], 
    event_group.attrs['ic_centroid'], 
    event_group.attrs['ic_integral'], 
    event_group.attrs['ic_multiplicity'], 
    est_params, 
    det_params, 
    results
)

Now we can look at some of our results! First we'll look at the circle fit, and the estimated vertex poistion, which is how we estimate $B\rho$. We'll also draw a line corresponding to the estimated initial direction. One important part of the estimation phase is smoothing. Smoothing splines are applied to the x, y, and charge coordinates as a function of z. So your cluster might look a little different than before, but that is by design. The smoothing helps us get accurate measures for important quantities like dEdx.

In [ ]:
if len(results["event"]) == 0:
    raise Exception("No results")

center_x = results['center_x'][-1]
center_y = results['center_y'][-1]
vertex_x = results['vertex_x'][-1]
vertex_y = results['vertex_y'][-1]
vertex_z = results['vertex_z'][-1]
theta = results['polar'][-1]
phi = results['azimuthal'][-1]
brho = results['brho'][-1]
dedx = results["dEdx"][-1]
rho_mm = brho/det_params.magnetic_field * 1000.0 * np.sin(theta)
print(f"Brho(T*m): {brho}")
print(f"Rho(mm): {rho_mm}")
print(f"dEdx: {dedx}")
print(f"Polar(deg):{theta * 180.0/np.pi}")
print(f"Azimuthal(deg):{phi * 180.0/np.pi}")
print(f"Direction: {results['direction'][-1]}")
print(f"Circle center x: {center_x} y: {center_y}")
print(f"Vertex z: {vertex_z} x: {vertex_x} y: {vertex_y} Cluster z start: {cluster.data[0, 2]}")
length_samples = np.linspace(1.0, 50.0, 50)
dir_x_samples = length_samples * np.cos(phi) + vertex_x
dir_y_samples = length_samples * np.sin(phi) + vertex_y

circle_points = generate_circle_points(center_x, center_y, rho_mm)
beam_region = generate_circle_points(0., 0., det_params.beam_region_radius)

fig = make_subplots(2,1,specs=[[{"type": "scene"}],[{"type": "xy"}]],row_heights=[0.25,0.75])
fig.add_trace(
    go.Scatter3d(
        x=cluster.data[:, 2], 
        y=cluster.data[:, 0], 
        z=cluster.data[:, 1], 
        mode="markers",
        marker= {
            "size": 3,
            "color": cluster.data[:, 3]
        }, 
        name=f"Cluster-3D"
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter3d(
        x=[cluster.data[0, 2]], 
        y=[cluster.data[0, 0]], 
        z=[cluster.data[0, 1]], 
        mode="markers",
        marker= {
            "size": 3,
            "color": "red"
        }, 
        name=f"Cluster-3D Start"
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter3d(
        x=[vertex_z], 
        y=[vertex_x], 
        z=[vertex_y], 
        mode="markers",
        marker= {
            "size": 3,
            "color": "green"
        }, 
        name=f"Vertex-3D"
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(x=cluster.data[:, 0], y=cluster.data[:, 1], mode="markers", marker={"color": cluster.data[:, 3]}, name="Cluster"),
    row=2,
    col=1
)
fig.add_trace(
    go.Scatter(x=[cluster.data[0, 0]], y=[cluster.data[0, 1]], mode="markers", marker={"color": "red"}, name="Cluster Start"),
    row=2,
    col=1
)
fig.add_trace(
    go.Scatter(x=circle_points[:, 0], y=circle_points[:, 1], mode="lines", name="Circle Fit"),
    row=2,
    col=1
)
fig.add_trace(
    go.Scatter(x=beam_region[:, 0], y=beam_region[:, 1], mode="lines", name="Beam Region"),
    row=2,
    col=1
)
fig.add_trace(
    go.Scatter(x=[vertex_x], y=[vertex_y], mode="markers", marker={"color": "green"}, name="Vertex"),
    row=2,
    col=1
)
fig.add_trace(
    go.Scatter(x=dir_x_samples, y=dir_y_samples, mode="lines", name="Initial Direction"),
    row=2,
    col=1
)
fig.update_layout(
    xaxis_title="X (mm)",
    yaxis_title="Y (mm)",
    xaxis_range=[-300.0,300.0],
    yaxis_range=[-300.0,300.0],
    scene = {
        # "xaxis_range": [0.0, 1000.0],
        "yaxis_range": [-300.0, 300.0],
        "zaxis_range": [-300.0, 300.0],
        "xaxis_title": "Z (mm)",
        "yaxis_title": "X (mm)",
        "zaxis_title": "Y (mm)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        }
    },
    width=900,
    height=1200
)
fig.show()

We can also start building up a particle id using $B\rho$ and $\frac{dE}{dx}$. If you run the three cells above this repeatedly, you'll see the plot below fill with data!

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=results["dEdx"], y=results["brho"], mode="markers", name="PID")
)
fig.update_layout(
    xaxis_title="dEdx",
    yaxis_title="B&#961;",
    xaxis_range=[0.0, 2.0e4],
    yaxis_range=[0.0, 3.0]
)
fig.show()

In this way you can examine and tune the parameters for the estimation phase. If you want to make complete particle ID plots, it is recommended to use the particle_id.ipynb rather than these notebooks, which are more for demonstration.